In [ ]:
!pip install git+https://github.com/huggingface/transformers.git
!pip install deepspeed==0.6.5
!pip install accelerate
!pip install sentencepiece
!pip install langchain
!pip install torch
!pip install bitsandbytes

In [1]:
import os
import deepspeed
import torch


[2023-08-23 16:18:02,286] [WARNING] [partition_parameters.py:60:<module>] unable to find torch.distributed._all_gather_base. will fall back to torch.distributed.all_gather which will result in suboptimal performance. please consider upgrading your pytorch installation.


In [ ]:
!pip uninstall torch -y
!pip install torch==1.5

In [5]:
print(torch.cuda.is_available())
print(torch.cuda.device_count())
print(torch.cuda.get_device_name(0))

True
4
NVIDIA GeForce GTX 1080 Ti


In [1]:
import torch
print(torch.__version__)


2.0.1+cu117


transformer --> deepseed
accelerate --> deepseed

In [9]:
from transformers import LlamaTokenizer, LlamaForCausalLM, pipeline


In [3]:
from langchain.llms import HuggingFacePipeline

In [4]:
import transformers
print(transformers.__version__)

4.30.0


[deeepspeed need torch 1, not install]

accelerate and others need torch 2
transformers==4.30

In [5]:
base_model = LlamaForCausalLM.from_pretrained(
    "chavinlo/alpaca-native",
    load_in_8bit=True,
    device_map='auto',
)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [6]:
import scipy

66% =>
/dev/sdb2       439G  282G  135G  68% /


In [7]:
tokenizer = LlamaTokenizer.from_pretrained("chavinlo/alpaca-native")

In [12]:
from langchain import PromptTemplate, LLMChain
template = """
Write a SQL Query given the table name {Table} and columns as a list {Columns} for the given question : 
{question}.
"""

prompt = PromptTemplate(template=template, input_variables=["Table","question","Columns"])

In [13]:
pipe = pipeline(
    "text-generation",
    model=base_model,
    tokenizer=tokenizer,
    max_length=500,
    temperature=0.3,
    top_p=0.95,
    repetition_penalty=1.2
)

local_llm = HuggingFacePipeline(pipeline=pipe)
llm_chain = LLMChain(prompt=prompt, llm=local_llm)

In [14]:
def get_llm_response(tble,question,cols):
    llm_chain = LLMChain(prompt=prompt, 
                         llm=local_llm
                         )
    response= llm_chain.run({"Table" : tble,"question" :question, "Columns" : cols})
    print(response)

In [ ]:
tble = "employee"
cols = ["id","name","date_of_birth","band","manager_id"]
question = "Query the count of employees in band L6 with 239045 as the manager ID"
get_llm_response(tble,question,cols)

In [16]:
get_llm_response(tble,question,cols)

Answer: SELECT COUNT(*) FROM employee WHERE band='L6' AND manager_id=239045;


In [17]:
question = "請查出管理員ID是23904 並在 band L6 的員工數量"
get_llm_response(tble,question,cols)

SELECT manager_id = 23904 AND count(employee) AS total_in_L6 FROM employee WHERE band = "L6";
